In [2]:
import rasterio
import geopandas as gpd
import numpy as np
import os

def generate_tiles(image_path, shapefile_path, tile_size=(2048, 2048), overlap=0.0, save_dir='tiles'):
    # Load the raster image
    with rasterio.open(image_path) as src:
        # image = src.read()
        image_profile = src.profile
        image_crs = src.crs

    # Load the shapefile
    shapefile = gpd.read_file(shapefile_path)

    # Ensure both the raster image and shapefile have the same CRS
    shapefile_crs = shapefile.crs
    if image_crs != shapefile_crs:
        shapefile = shapefile.to_crs(image_crs)

    # Create the save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Determine tile parameters
    stride = int(tile_size[0] * (1 - overlap))

    # Generate tiles
    tiles = []
    for window in rasterio.windows.sliding_windows(image.shape, tile_size, stride):
        # Read the tile from the image
        # tile = image[:, window[0], window[1]]
        tile = src.read(window=window)

        # Extract the tile bounds
        x_min, y_min, x_max, y_max = window[1].bounds

        # Create a new GeoDataFrame for the tile
        tile_shapefile = gpd.GeoDataFrame({'id': [1], 'geometry': [window[1]]}, crs=image_crs)

        # Clip the shapefile to the tile bounds
        clipped_shapefile = gpd.overlay(shapefile, tile_shapefile, how='intersection')

        # Create an empty mask
        mask = np.zeros(tile_size, dtype=np.uint8)

        # Create the mask based on the surface type
        for _, row in clipped_shapefile.iterrows():
            geometry = row.geometry
            description = row['DESCRIPTION']
            pixel_value = get_pixel_value_from_description(description)

            coords = rasterio.features.geometry_mask([geometry], out_shape=tile_size, transform=image_profile['transform'], invert=True)
            mask |= (coords * pixel_value).astype(np.uint8)

        tiles.append((tile, mask))

    # Save the tiles
    for i, (tile, mask) in enumerate(tiles):
        save_tile_path = os.path.join(save_dir, f'tile_{i}')
        save_image_path = f'{save_tile_path}_image.tif'
        save_mask_path = f'{save_tile_path}_mask.tif'

        # Create a new profile for the tile image
        tile_profile = image_profile.copy()
        tile_profile.update(width=tile.shape[2], height=tile.shape[1], transform=rasterio.windows.transform(window, image_profile['transform']))

        # Save the tile image
        with rasterio.open(save_image_path, 'w', **tile_profile) as dst:
            dst.write(tile)

        # Save the tile mask
        with rasterio.open(save_mask_path, 'w', **tile_profile) as dst:
            dst.write(mask, 1)

    print(f'Tiles saved in "{save_dir}" directory.')

def get_pixel_value_from_description(description):
    # Define a mapping from surface types to pixel values
    surface_mapping = {
        'Alley': 0,
        'Paved Drive': 1,
        'Parking Lot': 2,
        'Road': 3,
        'Intersection': 4,
        'Paved Median Island': 5,
        'Paved Traffic Island': 6,
        'Unpaved Traffic Island': 7,
        'Unpaved Median Island': 8,
        'Hidden Median': 9,
        'Hidden Road': 10
    }

    # Return the corresponding pixel value based on the description
    return surface_mapping.get(description, 0) # Default to 0 if description not found in mapping

image_path = '../data/OpenDataDC_Orthophoto_2021_JPEG2000/DCOCTO-2021.jp2'
shapefile_path = '../data/RoadsRep.geojson'
generate_tiles(image_path, shapefile_path, save_dir='../data/tiles')

AttributeError: module 'rasterio.windows' has no attribute 'sliding_windows'

In [ ]:
import rasterio
from rasterio.windows import Window
from shapely.geometry import shape

def generate_tiles(image_path, shapefile_path, tile_size):
    with rasterio.open(image_path) as src:
        crs = src.crs
        transform = src.transform

        with fiona.open(shapefile_path, "r") as shapefile:
            for feature in shapefile:
                geometry = shape(feature['geometry'])
                bounds = geometry.bounds

                left, bottom, right, top = bounds
                start_col, start_row = src.index(left, top)
                end_col, end_row = src.index(right, bottom)

                for row in range(start_row, end_row, tile_size):
                    for col in range(start_col, end_col, tile_size):
                        window = Window(col, row, tile_size, tile_size)
                        img_tile = src.read(window=window)
                        transform_tile = rasterio.windows.transform(window, transform)
                        tile_bounds = rasterio.windows.bounds(window, transform)

                        # Perform your desired operations on the image tile here
                        # You can access the corresponding shape attributes using feature['properties']

                        # Example: Saving the image tile
                        tile_name = f"tile_{row}_{col}.jp2"
                        tile_path = f"path/to/save/{tile_name}"
                        with rasterio.open(tile_path, 'w', driver='JP2', width=tile_size, height=tile_size,
                                           count=src.count, dtype=src.dtypes[0], crs=crs,
                                           transform=transform_tile) as tile_dst:
                            tile_dst.write(img_tile)

# Usage example
image_path = 'path/to/raster.jp2'
shapefile_path = 'path/to/shapefile.geojson'
tile_size = 512
generate_tiles(image_path, shapefile_path, tile_size)
